# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


KeyboardInterrupt: 

In [ ]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Disaster_Response_Table', engine)

#Create X and Y 
X = df["message"].values
Y = df.iloc[:,4:].values

#Retrieve number of labels and label names
num_of_labels = Y.shape[1]
label_names = df.columns[4:]

### 2. Write a tokenization function to process your text data

In [3]:
#Build tokenize function
def tokenize(text):
    """
    This tokenize function processes input text to generate useful word tokens
    Input: text content
    Output: processed word tokens    
    """
    #Define stop_words and create stemming and lemmatizing objects
    stop_words = stopwords.words("english")
    stemming = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    #First use regular expression to get rid of punctuations
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #Then tokenize the text and eliminate stop words
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    #In the last step, process the word token list with lemmitizer and then stemmer
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
   
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
#Import required libraries for Ml pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

#Create pipeline object
Original_pipeline = Pipeline(
[("Vectorizer",CountVectorizer(tokenizer = tokenize)),\
 ("TFIDF",TfidfTransformer()),\
 ("Estimator",MultiOutputClassifier(RandomForestClassifier(n_estimators = 100, min_samples_split = 3)))])
# 'Estimator__n_estimators': [50, 100, 200],
        #'Estimator__min_samples_split': [2, 3, 4]}

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Split X and Y into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
Original_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('Vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
#Report test set results
y_pred = Original_pipeline.predict(X_test)

Metrics_Original = pd.DataFrame(columns = ["accuracy", "precision", "recall", "f1-score"])

for i in range(num_of_labels):
    test = y_test[:,i]
    predict = y_pred[:,i]
    TP = np.sum(np.logical_and(test == 1, predict == 1))
    FN = np.sum(np.logical_and(test == 1, predict == 0))
    FP = np.sum(np.logical_and(test == 0, predict == 1))
    TN = np.sum(np.logical_and(test == 0, predict == 0))
    accuracy = (TP + TN)/(TP + TN + FP +FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2*(precision*recall)/(precision + recall)
    Metrics_Original = Metrics_Original.append({"accuracy": accuracy, "precision": precision, "recall": recall, "f1-score": f1_score}, ignore_index = True)

Metrics_Original.to_csv("Metrics_Original.csv")
Metrics_Original  

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in long_scalars
  from ipykernel import kernelapp as app


,accuracy,precision,recall,f1-score
0,0.830837,0.849345,0.949876,0.896802
1,0.894928,0.813309,0.494382,0.614955
2,0.995995,NaN,0.000000,NaN
3,0.792143,0.762271,0.726606,0.744011
4,0.920671,0.611940,0.095128,0.164659
5,0.953471,0.787879,0.098859,0.175676
6,0.974447,0.800000,0.057143,0.106667
7,0.981312,0.000000,0.000000,NaN
8,0.968345,0.555556,0.059524,0.107527
9,1.000000,NaN,NaN,NaN


### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
#Import GridSearch Library
from sklearn.model_selection import GridSearchCV

#Define grid search parameters
parameters = {
        'Vectorizer__ngram_range': ((1, 1), (1, 2)),
#        'Vectorizer__max_df': (0.5, 0.75, 1.0),
#        'Vectorizer__max_features': (None, 5000, 10000),
        'TFIDF__use_idf': (True, False),
        'Estimator__estimator__n_estimators': [50, 100, 200]}

cv = GridSearchCV(Original_pipeline, param_grid = parameters)

#Re-train the model with grid search result
#In order to save running time, we take only 
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X, Y,train_size = 0.20, test_size = 0.05)
cv.fit(X_train_cv, y_train_cv)

KeyboardInterrupt: 

NameError: name 'self' is not defined

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [81]:
#Report test set results after grid search
#for i in range(num_of_labels):
#    print("Classification Report of " + label_names[i])
#    print(classification_report(y_test[:,i], y_pred[:,i]))

#Report test set results
y_pred_cv = cv.predict(X_test_cv)

Metrics_cv = pd.DataFrame(columns = ["accuracy", "precision", "recall", "f1-score"])

for i in range(num_of_labels):
    test = y_test_cv[:,i]
    predict = y_pred_cv[:,i]
    TP = np.sum(np.logical_and(test == 1, predict == 1))
    FN = np.sum(np.logical_and(test == 1, predict == 0))
    FP = np.sum(np.logical_and(test == 0, predict == 1))
    TN = np.sum(np.logical_and(test == 0, predict == 0))
    accuracy = (TP + TN)/(TP + TN + FP +FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2*(precision*recall)/(precision + recall)
    Metrics_cv = Metrics_cv.append({"accuracy": accuracy, "precision": precision, "recall": recall, "f1-score": f1_score}, ignore_index = True)

Metrics_cv.to_csv("Metrics_cv.csv")
Metrics_cv

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [77]:
#Define new custom transformer called Special_Punc_Counter to get average amount of special punctuation per sentence
from sklearn.base import BaseEstimator, TransformerMixin
class Special_Puncs_Counter(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.use_counter = True
        self.get_params = self.get_params()
    #count_special_puncs method is the main method to count special pnctuations
    def get_params(self, deep=False):
        return {'use_counter':self.use_counter}
    def count_special_puncs(self, text):
        """
        This function analyzes the amount of special punctuations and divide it by number of sentences
        Imput: Text to analyze
        Output: Calculated special punctuation per sentence
        """
        sentence_list = nltk.sent_tokenize(text)
        num_sentence = len(sentence_list)
        count = 0
        for sentence in sentence_list:
            puncs = re.findall(r'[!?~<>({:;]',sentence)
            count += len(puncs)
        return count/num_sentence
    
    #Define fit method
    def fit(self, X, y=None):
        return self
    
    #Define transform method to transform series of interest
    def transform(self, X, use_counter = True):
        self.use_counter = use_counter
        X_tagged = pd.Series(X).apply(self.count_special_puncs)
        return pd.DataFrame(X_tagged)

In [70]:
A = Special_Puncs_Counter()
A.transform(X_test, use_counter = False)
Y = A.get_params

In [78]:
#Add Feature Union with new custom transformer to our pipeline
New_Feature_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_processing', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('count_special_puncs', Special_Puncs_Counter())
        ])),

        ('Estimator', RandomForestClassifier())
    ])


In [79]:
#Import GridSearch Library
from sklearn.model_selection import GridSearchCV

#Define grid search parameters
parameters = {
        'features__text_processing__Vectorizer__ngram_range': ((1, 1), (1, 2)),
#        'Vectorizer__max_df': (0.5, 0.75, 1.0),
#        'Vectorizer__max_features': (None, 5000, 10000),
        'features__text_processing__tfidf__use_idf': (True, False),
        'features__Estimator__estimator__n_estimators': [50, 100, 200]}

cv = GridSearchCV(New_Feature_pipeline, param_grid = parameters)

#Re-train the model with grid search result
#In order to save running time, we take only 
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X, Y,train_size = 0.20, test_size = 0.05)
cv.fit(X_train_cv, y_train_cv)

TypeError: 'dict' object is not callable

ValueError: too many values to unpack (expected 2)

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X[:1000], Y[:1000], test_size = 0.2)
New_Feature_pipeline.fit(X_train, y_train)
y_pred = cv.predict(X_test)
for i in range(num_of_labels):
    print("Classification Report of " + label_names[i])
    print(classification_report(y_test[:,i], y_pred[:,i]))

Classification Report of related
             precision    recall  f1-score   support

          0       0.94      0.87      0.90        38
          1       0.96      0.99      0.97       160
          2       0.00      0.00      0.00         2

avg / total       0.95      0.95      0.95       200

Classification Report of request
             precision    recall  f1-score   support

          0       0.92      0.94      0.93        89
          1       0.95      0.94      0.95       111

avg / total       0.94      0.94      0.94       200

Classification Report of offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       200

avg / total       1.00      1.00      1.00       200

Classification Report of aid_related
             precision    recall  f1-score   support

          0       0.86      1.00      0.92        77
          1       1.00      0.89      0.94       123

avg / total       0.94      0.94      0.94       200

Classi

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [129]:
import pickle
pickle_file = open("model.pkl",'wb')
pickle.dump(New_Feature_pipeline, pickle_file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.